http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code  
https://mc.ai/word2vec-for-phrases%E2%80%8A-%E2%80%8Alearning-embeddings-for-more-than-one-word/  
https://becominghuman.ai/how-does-word2vecs-skip-gram-work-f92e0525def4

# Word Embedding

* Are **methods to represent words in a numerical way**.
* Example: **one-hot encoding** to map each word to a one-hot vector.
<img src="figures/one-hot-word-embedding-vectors.png" width="60%">

# Idea behind Word2Vec
* The idea behind Word2Vec is pretty  
  `the meaning of a word can be inferred by the company it keeps`. 
* This is analogous to the saying,  
  “`show me your friends, and I’ll tell who you are`”. 

* If we have two words that have **very similar neighbors**,   
  then these words are probably quite **similar in meaning** or are at least highly related.

**Example** 

* the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

Viceversa
* the word “`play`” has a different meaning in the sentences  
  * “`The boy loves to play outside`”
  * “`The play was fantastic`”.

**Learning word from its context**

* `Word2Vec` starts from **large unsupervised corpus** 
  * **for each word** in the corpus, we try to **predict it** by its **given context** (CBOW), or 
  * trying to **predict the context given** a specific **word** (Skip-Gram)

<img src="figures/w2v-models.png" width="60%">

**Neural network structure**

Word2vec is a neural network structure **to generate word embedding** by training the model on a supervised classification problem. 

* Introduced by **Mikolov** et al.,2013 in the paper *Efficient Estimation of Word Representations in Vector Space*  
* used to measure syntactic and **semantic similarities** between words.

**Word2Vec with Continuous Bag of Words (CBOW) Learning technique**

<img src="figures/Architecture-of-Word2Vec-with-CBOW-technique.png" width="40%">

**Word2vec SKIP-GRAM architecture**

Given a word $w(t)$, the model **predicts the words** that precede and proceed it in a **window** of 4 words, 
$w(t-2)$, $w(t-1)$, $w(t+1)$, $w(t+2)$ 
<img src="figures/WORD2VEC-SKIP-GRAM.png" width="70%">

**Training SKIP-GRAM**

<img src="figures/skip-gram-training.png" width="70%">

**The Neural model for Skip-Gram**

<img src="figures/skip-gram-neural-model.png" width="70%">

**Emdedding matrix after Word2Vec training**

* Each **word** will be represented by a **d-dimension continuous vector**, 
* The **meaning of each word** will be captured by its **relation to other words**. 

**The reason**
* in training time, `if two target words share the some context, intuitively the weight of the network for this two target words will be close to each other`. 

**Similarity tasks**

Looking the vector as a whole, one can perform many similarity tasks.   
For example, 
* we get that $$V(“King”)-V(“Man”)+V(“Woman) ~= V(“Queen”)$$ and $$V(“Paris”)-V(“France)+V(“Spain”) ~= V(“Madrid”)$$. 
* We can perform similarity measures, like **cosine-similarity between the vectors** and get that   
  “`president`” will be close to “`Obame`”, “`Trump`”, “`CEO`”, “`Chairman`”, etc.

# Word2Vec in Gensim 

**Performance** is really a combination of two things, 
1. your **input data** 
2. your **parameter** settings. 

**Note** that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [6]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
**The secret** to getting Word2Vec really working for you is to **have lots and lots of text data**. 
* We are going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. 
* This dataset has full user **reviews of cars and hotels**. 

**Download** dataset from https://github.com/kavgan/nlp-in-practice/blob/master/word2vec/reviews_data.txt.gz (86 MB)

In [7]:
data_file="data/reviews_data.txt.gz"
# let's take a closer look at this data below by printing the first line.
with gzip.open (data_file, 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
We can read it into a list so that we can pass this on to the Word2Vec model. 

**Notice** 
* we are directly reading the compressed file. 
* We are doing a basic **pre-processing** of the reviews using `gensim.utils.simple_preprocess (line)`.   
  * such as **tokenization**, **lowercasing**, etc.. and 
  * returns back a **list of tokens** (words).

In [4]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2019-05-21 09:02:45,609 : INFO : reading file data/reviews_data.txt.gz...this may take a while
2019-05-21 09:02:45,612 : INFO : read 0 reviews
2019-05-21 09:02:47,564 : INFO : read 10000 reviews
2019-05-21 09:02:49,533 : INFO : read 20000 reviews
2019-05-21 09:02:51,772 : INFO : read 30000 reviews
2019-05-21 09:02:53,883 : INFO : read 40000 reviews
2019-05-21 09:02:56,196 : INFO : read 50000 reviews
2019-05-21 09:02:58,481 : INFO : read 60000 reviews
2019-05-21 09:03:00,407 : INFO : read 70000 reviews
2019-05-21 09:03:03,078 : INFO : read 80000 reviews
2019-05-21 09:03:05,139 : INFO : read 90000 reviews
2019-05-21 09:03:07,566 : INFO : read 100000 reviews
2019-05-21 09:03:09,335 : INFO : read 110000 reviews
2019-05-21 09:03:11,167 : INFO : read 120000 reviews
2019-05-21 09:03:12,999 : INFO : read 130000 reviews
2019-05-21 09:03:15,232 : INFO : read 140000 reviews
2019-05-21 09:03:17,410 : INFO : read 150000 reviews
2019-05-21 09:03:19,344 : INFO : read 160000 reviews
2019-05-21 09:03:2

## Training the Word2Vec model

* Just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). 
* Word2Vec uses all these tokens to internally **create a vocabulary**.
* call `train(...)` to start training the Word2Vec model. (takes about 10 minutes)
* we are **not** going to **use the neural network** after training. 
* **The goal** is to learn the weights of the hidden layer, i.e. the **word vectors** that we’re trying to learn. 

In [5]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-05-21 09:04:24,546 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-05-21 09:04:24,548 : INFO : collecting all words and their counts
2019-05-21 09:04:24,549 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-21 09:04:24,840 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-05-21 09:04:25,110 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-05-21 09:04:25,426 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-05-21 09:04:25,736 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-05-21 09:04:26,074 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-05-21 09:04:26,399 : INFO : PROGRESS: at sentence #60000, processed 11013723 words, keeping 76780 word types
2019-05-21 09:04:26,679 : INFO : PROGRESS: a

2019-05-21 09:05:11,582 : INFO : EPOCH 1 - PROGRESS: at 82.25% examples, 671674 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:05:12,611 : INFO : EPOCH 1 - PROGRESS: at 83.54% examples, 663832 words/s, in_qsize 18, out_qsize 2
2019-05-21 09:05:13,617 : INFO : EPOCH 1 - PROGRESS: at 85.21% examples, 660545 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:05:14,629 : INFO : EPOCH 1 - PROGRESS: at 87.71% examples, 661326 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:05:15,632 : INFO : EPOCH 1 - PROGRESS: at 90.25% examples, 662542 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:05:16,636 : INFO : EPOCH 1 - PROGRESS: at 92.57% examples, 662669 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:05:17,637 : INFO : EPOCH 1 - PROGRESS: at 94.93% examples, 663534 words/s, in_qsize 19, out_qsize 1
2019-05-21 09:05:18,643 : INFO : EPOCH 1 - PROGRESS: at 97.39% examples, 664414 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:05:19,652 : INFO : EPOCH 1 - PROGRESS: at 99.76% examples, 664720 words/s,

2019-05-21 09:06:08,453 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-21 09:06:08,454 : INFO : EPOCH - 2 : training on 41519355 raw words (30346660 effective words) took 48.7s, 622921 effective words/s
2019-05-21 09:06:09,465 : INFO : EPOCH 3 - PROGRESS: at 2.00% examples, 625457 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:06:10,484 : INFO : EPOCH 3 - PROGRESS: at 4.15% examples, 631103 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:06:11,504 : INFO : EPOCH 3 - PROGRESS: at 6.38% examples, 646614 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:06:12,504 : INFO : EPOCH 3 - PROGRESS: at 8.57% examples, 655624 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:06:13,511 : INFO : EPOCH 3 - PROGRESS: at 10.53% examples, 667255 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:06:14,516 : INFO : EPOCH 3 - PROGRESS: at 12.28% examples, 667993 words/s, in_qsize 16, out_qsize 3
2019-05-21 09:06:15,528 : INFO : EPOCH 3 - PROGRESS: at 14.46% examples, 676151 words/s, 

2019-05-21 09:07:12,228 : INFO : EPOCH 4 - PROGRESS: at 34.53% examples, 613865 words/s, in_qsize 20, out_qsize 0
2019-05-21 09:07:13,238 : INFO : EPOCH 4 - PROGRESS: at 36.53% examples, 611231 words/s, in_qsize 16, out_qsize 3
2019-05-21 09:07:14,253 : INFO : EPOCH 4 - PROGRESS: at 39.03% examples, 615444 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:07:15,301 : INFO : EPOCH 4 - PROGRESS: at 41.44% examples, 616559 words/s, in_qsize 18, out_qsize 4
2019-05-21 09:07:16,293 : INFO : EPOCH 4 - PROGRESS: at 43.79% examples, 618302 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:07:17,302 : INFO : EPOCH 4 - PROGRESS: at 46.14% examples, 618752 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:07:18,369 : INFO : EPOCH 4 - PROGRESS: at 48.31% examples, 618400 words/s, in_qsize 20, out_qsize 1
2019-05-21 09:07:19,366 : INFO : EPOCH 4 - PROGRESS: at 50.64% examples, 620052 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:07:20,390 : INFO : EPOCH 4 - PROGRESS: at 52.55% examples, 617792 words/s,

2019-05-21 09:08:16,614 : INFO : EPOCH 5 - PROGRESS: at 75.07% examples, 665268 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:08:17,650 : INFO : EPOCH 5 - PROGRESS: at 77.31% examples, 666142 words/s, in_qsize 20, out_qsize 0
2019-05-21 09:08:18,655 : INFO : EPOCH 5 - PROGRESS: at 79.54% examples, 666723 words/s, in_qsize 17, out_qsize 2
2019-05-21 09:08:19,693 : INFO : EPOCH 5 - PROGRESS: at 81.85% examples, 667090 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:08:20,709 : INFO : EPOCH 5 - PROGRESS: at 84.19% examples, 667814 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:08:21,720 : INFO : EPOCH 5 - PROGRESS: at 86.39% examples, 667816 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:08:22,726 : INFO : EPOCH 5 - PROGRESS: at 88.84% examples, 667819 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:08:23,729 : INFO : EPOCH 5 - PROGRESS: at 91.26% examples, 668399 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:08:24,730 : INFO : EPOCH 5 - PROGRESS: at 93.45% examples, 668083 words/s,

...

2019-05-21 09:12:13,042 : INFO : EPOCH 6 - PROGRESS: at 1.93% examples, 605917 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:12:14,055 : INFO : EPOCH 6 - PROGRESS: at 4.11% examples, 630606 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:12:15,056 : INFO : EPOCH 6 - PROGRESS: at 6.21% examples, 636422 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:12:16,060 : INFO : EPOCH 6 - PROGRESS: at 8.08% examples, 624313 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:12:17,080 : INFO : EPOCH 6 - PROGRESS: at 9.80% examples, 616162 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:12:18,086 : INFO : EPOCH 6 - PROGRESS: at 11.44% examples, 613580 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:12:19,095 : INFO : EPOCH 6 - PROGRESS: at 13.17% examples, 614643 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:12:20,104 : INFO : EPOCH 6 - PROGRESS: at 14.81% examples, 608153 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:12:21,118 : INFO : EPOCH 6 - PROGRESS: at 16.51% examples, 605339 words/s, in_q

2019-05-21 09:13:17,064 : INFO : EPOCH 7 - PROGRESS: at 39.44% examples, 657002 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:13:18,072 : INFO : EPOCH 7 - PROGRESS: at 42.14% examples, 660250 words/s, in_qsize 20, out_qsize 1
2019-05-21 09:13:19,098 : INFO : EPOCH 7 - PROGRESS: at 44.70% examples, 661620 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:13:20,110 : INFO : EPOCH 7 - PROGRESS: at 46.99% examples, 661308 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:13:21,127 : INFO : EPOCH 7 - PROGRESS: at 49.47% examples, 662448 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:13:22,131 : INFO : EPOCH 7 - PROGRESS: at 51.42% examples, 658725 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:13:23,153 : INFO : EPOCH 7 - PROGRESS: at 53.49% examples, 658016 words/s, in_qsize 19, out_qsize 1
2019-05-21 09:13:24,153 : INFO : EPOCH 7 - PROGRESS: at 55.91% examples, 658590 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:13:25,182 : INFO : EPOCH 7 - PROGRESS: at 58.38% examples, 660229 words/s,

2019-05-21 09:14:21,379 : INFO : EPOCH 8 - PROGRESS: at 81.43% examples, 663859 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:14:22,380 : INFO : EPOCH 8 - PROGRESS: at 83.76% examples, 664546 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:14:23,388 : INFO : EPOCH 8 - PROGRESS: at 85.96% examples, 664856 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:14:24,407 : INFO : EPOCH 8 - PROGRESS: at 88.51% examples, 665605 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:14:25,455 : INFO : EPOCH 8 - PROGRESS: at 90.95% examples, 665516 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:14:26,467 : INFO : EPOCH 8 - PROGRESS: at 93.28% examples, 665946 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:14:27,484 : INFO : EPOCH 8 - PROGRESS: at 95.32% examples, 663659 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:14:28,527 : INFO : EPOCH 8 - PROGRESS: at 97.68% examples, 663954 words/s, in_qsize 19, out_qsize 3
2019-05-21 09:14:29,367 : INFO : worker thread finished; awaiting finish of 9 more threa

2019-05-21 09:15:17,751 : INFO : EPOCH 10 - PROGRESS: at 6.26% examples, 635167 words/s, in_qsize 18, out_qsize 1
2019-05-21 09:15:18,761 : INFO : EPOCH 10 - PROGRESS: at 8.53% examples, 650906 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:15:19,764 : INFO : EPOCH 10 - PROGRESS: at 10.35% examples, 655333 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:15:20,785 : INFO : EPOCH 10 - PROGRESS: at 12.18% examples, 659946 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:15:21,831 : INFO : EPOCH 10 - PROGRESS: at 14.24% examples, 661114 words/s, in_qsize 19, out_qsize 2
2019-05-21 09:15:22,859 : INFO : EPOCH 10 - PROGRESS: at 16.29% examples, 664434 words/s, in_qsize 20, out_qsize 1
2019-05-21 09:15:23,874 : INFO : EPOCH 10 - PROGRESS: at 18.11% examples, 663997 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:15:24,877 : INFO : EPOCH 10 - PROGRESS: at 19.93% examples, 666899 words/s, in_qsize 19, out_qsize 0
2019-05-21 09:15:25,883 : INFO : EPOCH 10 - PROGRESS: at 21.99% examples, 668255 w

(303490881, 415193550)

## Now, let's look at some output 
* a simple case of looking up **words similar to** the word `dirty`. 
* All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. 
* This returns the top 10 similar words. 

In [12]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2018-01-26 23:03:42,416 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.871721625328064),
 ('stained', 0.7922376990318298),
 ('unclean', 0.7915753126144409),
 ('dusty', 0.7772612571716309),
 ('smelly', 0.7618112564086914),
 ('grubby', 0.7483716011047363),
 ('dingy', 0.7330487966537476),
 ('gross', 0.7239381074905396),
 ('grimy', 0.7228356599807739),
 ('disgusting', 0.7213647365570068)]

**Another example**

Let's look at similarity for `polite`, `france` and `shocked`. 

In [50]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9174547791481018),
 ('friendly', 0.8309274911880493),
 ('cordial', 0.7990915179252625),
 ('professional', 0.7945970892906189),
 ('attentive', 0.7732747197151184),
 ('gracious', 0.7469891309738159)]

In [53]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('canada', 0.6603403091430664),
 ('germany', 0.6510637998580933),
 ('spain', 0.6431018114089966),
 ('barcelona', 0.61174076795578),
 ('mexico', 0.6070996522903442),
 ('rome', 0.6065913438796997)]

In [54]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.80775386095047),
 ('amazed', 0.7797470092773438),
 ('astonished', 0.7748459577560425),
 ('dismayed', 0.7680633068084717),
 ('stunned', 0.7603034973144531),
 ('appalled', 0.7466776371002197)]

You can even 
* specify **several positive examples** to get things that are related in the provided context 
* provide **negative examples** to say what should not be considered as related. 

In the example below we are asking for all items that **relate to bed** only:

In [55]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.7086508274078369),
 ('blanket', 0.7016597390174866),
 ('mattress', 0.7002605199813843),
 ('quilt', 0.6868821978569031),
 ('matress', 0.6777950525283813),
 ('pillowcase', 0.6413239240646362),
 ('sheets', 0.6382123827934265),
 ('foam', 0.6322235465049744),
 ('pillows', 0.6320573687553406),
 ('comforter', 0.5972476601600647)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the **similarity between two words** that are present in the vocabulary. 

In [57]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.76181122646029453

In [58]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0000000000000002

In [59]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.25355593501920781

**Under the hood**

the above three snippets computes the **cosine similarity** between the two specified words using word vectors of each. 
* From the scores, it makes sense that `dirty` is highly similar to `smelly` but 
* `dirty` is dissimilar to `clean`. 
* If you do a **similarity between two identical words**, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find odd items
You can even use Word2Vec to find odd items *given a list of items*.

In [63]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [77]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```


### `size`
The size of the **dense vector** to `represent each token` or word. 
* If you have **very limited data**, then size should be a much **smaller** value. 
* If you have **lots of data**, its good to experiment with **various sizes**. 

### `window`
The maximum distance between the target word and its neighboring word. 
* If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. 
* In theory, a smaller window should give you terms that are more related. 
* If you have **lots of data**, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. 
* The model would **ignore words that do not statisfy the** `min_count`. 
* Extremely **infrequent words are** usually **unimportant**, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. 
* Imagine if you need to **build a sentiment lexicon**. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 
* You could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could **find tags that are related to a given tag** and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
